In [2]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time

# Enable interactive plot window
plt.style.use('fivethirtyeight')
%matplotlib qt

# Function to load and preprocess data
def load_and_preprocess_data():
    try:
        # Load the data
        data = pd.read_csv('sales_report_real_time.csv')
        
        # Check if essential columns are present
        required_columns = ['Time', 'Revenue (MYR)']
        if not all(column in data.columns for column in required_columns):
            raise ValueError("Missing essential columns in data.")
        
        # Remove irrelevant columns
        data_cleaned = data.drop(columns=['Order ID', 'Customer ID', 'Product ID', 'Product Name', 'Product Category', 'Order Status'], errors='ignore')
        
        # Convert 'Time' to datetime and handle parsing errors
        data_cleaned['Time'] = pd.to_datetime(data_cleaned['Time'], format='%H:%M:%S', errors='coerce')
        
        # Drop rows with invalid 'Time' entries after coercion
        data_cleaned = data_cleaned.dropna(subset=['Time'])
        
        # Convert 'Time' to seconds for easier plotting
        data_cleaned['TimeInSeconds'] = data_cleaned['Time'].dt.hour * 3600 + data_cleaned['Time'].dt.minute * 60 + data_cleaned['Time'].dt.second

        # Handle missing values in 'Revenue (MYR)' by filling with zero
        data_cleaned['Revenue (MYR)'] = data_cleaned['Revenue (MYR)'].fillna(0)
        
        return data_cleaned
    except FileNotFoundError:
        print("Error: The file 'sales_report_real_time.csv' was not found.")
    except ValueError as ve:
        print(f"Value error encountered: {ve}")
    except Exception as e:
        print(f"Unexpected error in load_and_preprocess_data: {e}")
    return pd.DataFrame()  # Return empty DataFrame if there's an issue

# Perform EDA if data is loaded successfully
data_cleaned = load_and_preprocess_data()

# Real-time animation synchronized with the Excel Time column
start_time = time.time()  # Record the real-time start point

def animate(i):
    try:
        # Reload data in case it's being updated in real-time
        data = load_and_preprocess_data()
        
        if data.empty:
            print("No data to plot. Ensure 'sales_report_real_time.csv' exists and is correctly formatted.")
            return
        
        # Get elapsed real-time seconds since the script started
        elapsed_time = int(time.time() - start_time)
        
        # Filter data for the elapsed time
        data_filtered = data[data['TimeInSeconds'] <= elapsed_time]
        
        # Group by 'TimeInSeconds' and sum up the revenue for the same timestamps
        data_grouped = data_filtered.groupby('TimeInSeconds').agg({'Revenue (MYR)': 'sum'}).reset_index()

        # Clear the current axes
        plt.cla()

        # Plot the total revenue as a line chart
        x = data_grouped['TimeInSeconds']
        y = data_grouped['Revenue (MYR)']

        plt.plot(x, y, label='Total Revenue (MYR)', color='violet', marker='o', linestyle='-', alpha=0.8)

        # Set labels and layout
        plt.xlabel('Time (Seconds)')
        plt.ylabel('Revenue (MYR)')
        plt.title('Real-Time Total Revenue')
        plt.legend(loc='upper left')
        
        # Adjust layout to ensure labels are visible
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.15, left=0.08)  # Add padding to prevent cutoff
    except Exception as e:
        print(f"Unexpected error during animation: {e}")

# Create animation for real-time revenue plot
ani = FuncAnimation(plt.gcf(), animate, interval=1000, cache_frame_data=False)

plt.show()